#1-1

MNIST 데이터셋을 사용하여 간단한 GAN을 구현한 코드입니다.

코드를 실행시키고, 주석을 달아주세요.

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image


In [3]:
num_epochs = 100
batch_size = 100
learning_rate = 0.0002
img_size = 28 * 28
noise_size = 100
hidden_size1 = 256
hidden_size2 = 512
hidden_size3 = 1024
dir_name = "GAN_results"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if not os.path.exists(dir_name):
    os.makedirs(dir_name)

transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 텐서로 변환
    transforms.Normalize([0.5], [0.5]) # 이미지 정규화
])

# MNIST 데이터셋 로드
MNIST_dataset = datasets.MNIST(root='../../data/',
                               train=True,
                               transform=transform,
                               download=True)

# DataLoader 설정
data_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 20777716.10it/s]


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 628098.44it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 5654636.12it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2917832.56it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential( # sequential 모델 : 입력을 여러 계층에 순차적으로 전달
            nn.Linear(img_size, hidden_size3),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size3, hidden_size2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size2, hidden_size1),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_size1, 1),
            nn.Sigmoid()
        )

    def forward(self, x): # 모델의 순전파
        return self.model(x)

class Generator(nn.Module): # 노이즈로부터 이미지를 생성하는 모델
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, hidden_size3),
            nn.ReLU(),
            nn.Linear(hidden_size3, img_size),
            nn.Tanh()
        )

    def forward(self, x): # 순전파
        return self.model(x)

discriminator = Discriminator().to(device) # 모델 인스턴스 생성하고 이동
generator = Generator().to(device)

criterion = nn.BCELoss() # 손실함수 정의
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate) # 파라미터를 업데이트 할 옵티마이저 정의
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)


In [5]:
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        real_labels = torch.ones(batch_size, 1).to(device) # 진짜 이미지에 대한 레이블 생성
        fake_labels = torch.zeros(batch_size, 1).to(device) # 가짜 이미지에 대한 레이블 생성

        real_images = images.reshape(batch_size, -1).to(device) # 이미지 크기 변경 후 저장


        g_optimizer.zero_grad() # 기울기 초기화

        z = torch.randn(batch_size, noise_size).to(device) # 랜덤한 노이즈 벡터 생성
        fake_images = generator(z) # 노이지를 입력받아 가짜 이미지 생성

        g_loss = criterion(discriminator(fake_images), real_labels) # 가짜 이미지를 실제라고 예측하도록 유도
        g_loss.backward() # 제너레이터 손실에 대해 역전파
        g_optimizer.step() # 업데이트

        d_optimizer.zero_grad() # 기울기 초기화

        z = torch.randn(batch_size, noise_size).to(device) # 다시 가짜 이미지 생성
        fake_images = generator(z)

        real_loss = criterion(discriminator(real_images), real_labels) # 진짜 이미지를 진짜로 예측하도록 학습
        fake_loss = criterion(discriminator(fake_images.detach()), fake_labels) # 가짜 이미지를 가짜로 예측하도록 학습
        d_loss = (real_loss + fake_loss) / 2 # 손실 평균을 discriminator 손실로 설정

        d_loss.backward() # 손실에 대해 역전파
        d_optimizer.step() # 업데이트

        # 학습 진행상황 출력
        if (i + 1) % 150 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    save_image(fake_images.view(batch_size, 1, 28, 28),
               os.path.join(dir_name, f'GAN_fake_image_{epoch + 1}.png'))

    # discriminator가 진짜와 가짜 이미지를 얼마나 잘 구분하는지 평가
    d_performance = discriminator(real_images).mean().item()
    g_performance = discriminator(fake_images).mean().item()
    print(f"---------Epoch [{epoch + 1}/{num_epochs}] : D Performance: {d_performance:.2f}, G Performance: {g_performance:.2f}")

Epoch [1/100], Step [150/600], D Loss: 0.0430, G Loss: 3.1124
Epoch [1/100], Step [300/600], D Loss: 0.0533, G Loss: 14.3516
Epoch [1/100], Step [450/600], D Loss: 0.0133, G Loss: 6.7415
Epoch [1/100], Step [600/600], D Loss: 0.0118, G Loss: 11.3472
---------Epoch [1/100] : D Performance: 0.99, G Performance: 0.00
Epoch [2/100], Step [150/600], D Loss: 0.0008, G Loss: 9.9950
Epoch [2/100], Step [300/600], D Loss: 0.0017, G Loss: 6.5479
Epoch [2/100], Step [450/600], D Loss: 0.0008, G Loss: 11.2755
Epoch [2/100], Step [600/600], D Loss: 0.0649, G Loss: 17.5674
---------Epoch [2/100] : D Performance: 0.97, G Performance: 0.01
Epoch [3/100], Step [150/600], D Loss: 0.0471, G Loss: 11.3675
Epoch [3/100], Step [300/600], D Loss: 0.2561, G Loss: 5.0783
Epoch [3/100], Step [450/600], D Loss: 0.8017, G Loss: 6.5744
Epoch [3/100], Step [600/600], D Loss: 0.2068, G Loss: 10.9645
---------Epoch [3/100] : D Performance: 0.89, G Performance: 0.04
Epoch [4/100], Step [150/600], D Loss: 1.1512, G Los

#1-2

아래 마크다운으로 GAN_fake_image_1.png와 GAN_fake_image_100.png를 함께 첨부해주세요.

![GAN_fake_image](GAN_fake_image_1.png)
![GAN_fake_image](GAN_fake_image_100.png)